In [ ]:
!ssh

In [ ]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random
import numpy as np


In [ ]:
key = random.PRNGKey(0)
x = random.normal(key, (10,))
print(x)

In [ ]:
size = 3000
x = random.normal(key, (size, size), dtype=jnp.float32)
%timeit jnp.dot(x, x.T).block_until_ready()  # runs on the GPU

In [ ]:
from jax import device_put

x = np.random.normal(size=(size, size)).astype(np.float32)
x = device_put(x)
%timeit jnp.dot(x, x.T).block_until_ready()

In [ ]:
%timeit jnp.dot(x, x.T).block_until_ready()

In [ ]:
def selu(x, alpha=1.67, lmbda=1.05):
  return lmbda * jnp.where(x > 0, x, alpha * jnp.exp(x) - alpha)

x = random.normal(key, (1000000,))
%timeit selu(x).block_until_ready()

In [ ]:
selu_jit = jit(selu)
%timeit selu_jit(x).block_until_ready()

In [ ]:
def sum_logistic(x):
  return jnp.sum(1.0 / (1.0 + jnp.exp(-x)))

x_small = jnp.arange(3.)
derivative_fn = grad(sum_logistic)
print(derivative_fn(x_small))

In [ ]:
def first_finite_differences(f, x):
  eps = 1e-3
  return jnp.array([(f(x + eps * v) - f(x - eps * v)) / (2 * eps)
                   for v in jnp.eye(len(x))])


print(first_finite_differences(sum_logistic, x_small))

In [ ]:
print(grad(jit(grad(jit(grad(sum_logistic)))))(1.0))

In [ ]:
mat = random.normal(key, (150, 100))
batched_x = random.normal(key, (10, 100))

def apply_matrix(v):
  return jnp.dot(mat, v)

In [ ]:
def naively_batched_apply_matrix(v_batched):
  return jnp.stack([apply_matrix(v) for v in v_batched])

print('Naively batched')
%timeit naively_batched_apply_matrix(batched_x).block_until_ready()

In [ ]:
@jit
def batched_apply_matrix(v_batched):
  return jnp.dot(v_batched, mat.T)

print('Manually batched')
%timeit batched_apply_matrix(batched_x).block_until_ready()

In [ ]:
@jit
def vmap_batched_apply_matrix(v_batched):
  return vmap(apply_matrix)(v_batched)

print('Auto-vectorized with vmap')
%timeit vmap_batched_apply_matrix(batched_x).block_until_ready()

In [ ]:
vmap(apply_matrix,(0,),0)(random.normal(key, ( 10, 100))).shape

In [ ]:
help(vmap)

In [ ]:
xs = jnp.arange(3. * 4.).reshape(3, 4)
xs

In [ ]:
from jax import lax
print(vmap(lambda x: lax.psum(x, 'i'), axis_name='i')(xs))

In [ ]:
help(lax.psum)

In [ ]:
jax.pmap(lambda x: x ** 2)(jnp.arange(1)) 

In [ ]:
import jax.numpy as jnp

x_jnp = jnp.linspace(0, 10, 1000)
y_jnp = 2 * jnp.sin(x_jnp) * jnp.cos(x_jnp)
plt.plot(x_jnp, y_jnp);

In [ ]:
type(x_jnp)

In [ ]:
y = x_jnp.at[0].set(10)
print(x_jnp)
print(y)

In [ ]:
x = jnp.array([1, 2, 1])
y = jnp.ones(10)
jnp.convolve(x, y)

In [ ]:
import jax.numpy as jnp

def norm(X):
  X = X - X.mean(0)
  return X / X.std(0)

In [ ]:
from jax import jit
norm_compiled = jit(norm)

In [ ]:
np.random.seed(1701)
X = jnp.array(np.random.rand(10000, 10))
np.allclose(norm(X), norm_compiled(X), atol=1E-6)

In [ ]:
%timeit norm(X).block_until_ready()
%timeit norm_compiled(X).block_until_ready()

In [ ]:
from functools import partial

# @partial(jit, static_argnums=(1,))
def f(x, neg):
  return -x if neg else x

%timeit f(jnp.array([1,2]), True)

In [ ]:
from functools import partial

@partial(jit, static_argnums=(1,))
def f(x, neg):
  return -x if neg else x

%timeit -n1 -r1 f(1, False)

In [ ]:
%timeit -n1 -r1 f(1, False)

In [ ]:
%timeit -n1 -r1 f(1, True)

In [ ]:
%timeit -n1 -r1 f(1, True)

In [ ]:
from jax import jit
import jax.numpy as jnp
import numpy as np

@jit
def f(x):
  #return x.reshape((np.prod(x.shape),))
    return x[x<0]

f(x)

In [ ]:
x = np.arange(10)
@jit
def func(i):
    return jnp.asarray(x)[i]

%timeit -r1 -n1 func(jnp.arange(4))  

In [ ]:
%timeit -r1 -n1 func(jnp.arange(4))  

In [ ]:
x = jnp.arange(10)
@jit
def func(x,):
    return jnp.split(x, 2, 0)

func(x,)
%timeit func(x, )

In [ ]:
grad(func)(x)

In [ ]:
x = jnp.arange(10)
# @partial(jit, static_argnums=1)
def func(x, axis):
    return jnp.split(x, 2, axis)

func(x, 0)
%timeit func(x, 0)

In [ ]:
from jax import jit
import jax.numpy as jnp
import numpy as np

@jit
def f(x):
  return x.reshape((np.prod(x.shape),))

x = jnp.ones((2, 3))
f(x)

In [ ]:
x = jnp.ones((2, 4,6))
%timeit -n100 -r1 f(x)

In [ ]:
x2=np.array([[1,2],[3,4]])
%timeit -n100 -r1  f(x2)

In [ ]:
import jax.numpy as jnp
import jax
from jax import jit

def slow_f(x):
  # Element-wise ops see a large benefit from fusion
  return x * x + x * 2.0

x = jnp.ones((5000, 5000))
fast_f = jax.jit(slow_f)         # 静态编译slow_f;
%timeit -n10 -r3 fast_f(x)
%timeit -n10 -r3 slow_f(x)

In [ ]:
jaxpr = jax.make_jaxpr(fast_f)
jaxpr(3)

In [ ]:
def unjitted_loop_body(prev_i):
  return prev_i + 1

def g_inner_jitted_lambda(x):
  i = 0
  while i < 20:
    # Don't do this!, lambda will return
    # a function with a different hash
    i = jax.jit(lambda x: unjitted_loop_body(x))(i)
  return x + i

def g_inner_jitted_normal(x):
  i = 0
  while i < 20:
    # this is OK, since JAX can find the
    # cached, compiled function
    i = jax.jit(unjitted_loop_body)(i)
  return x + i

print("jit called in a loop with lambdas:")
%timeit -n 10 g_inner_jitted_lambda(10)

print("jit called in a loop with caching:")
%timeit -n 10 g_inner_jitted_normal(10)

In [ ]:
def f(x):
    if x < 3:
        return 3. * x ** 2
    else:
        return -4 * x
    
static_f = jax.jit(f, static_argnums=(0,))
# jax.make_jaxpr(static_f, static_argnums=(0,))(2)

In [ ]:
%timeit static_f(2)

In [ ]:
%timeit static_f(4)

In [ ]:
static_f(2),static_f(4)

In [ ]:
from jax import jit
import jax.numpy as jnp
import numpy as np

# x = np.random.randn(3, 4)


@jit
def f(x):
  x = np.prod(x.shape)
  print(x)
  return x
#   return x.reshape()

x = jnp.ones((2, 3))
%timeit -r1 -n1 f(x)
%timeit -r1 -n1 f(x)
# jax.make_jaxpr(f)(x)

In [ ]:
x = jnp.ones((2, 4))
%timeit -r1 -n1 f(x)
%timeit -r1 -n1 f(x)

In [ ]:
x = jnp.ones((5, 14))
%timeit -n1 -r1 f(x)
%timeit -n1 -r1 f(x)

In [ ]:
import numpy as np
from jax import grad, jit
from jax import lax
from jax import random
import jax
import jax.numpy as jnp

In [ ]:
@jit
def f(x):
  #return x.reshape(jnp.array(x.shape).prod())
    return jnp.array(x.shape).prod()

x = jnp.ones((2, 3))
f(x)
%timeit f(x)

In [ ]:
@jit
def impure_print_side_effect(x):
  print("Executing function")  # This is a side-effect
  return x
print ("First call: ", (impure_print_side_effect)(4.))

In [ ]:
print ("Second call: ", (impure_print_side_effect)(5.))

In [ ]:
print ("Third call, different type: ", (impure_print_side_effect)(jnp.array([5.])))

In [ ]:
g = 0.
def impure_saves_global(x):
  global g
  g = x
  print('x',x)
  return x

# JAX runs once the transformed function with special Traced values for arguments
print ("First call: ", jit(impure_saves_global)(4.))
print ("Saved global: ", g)  # Saved global has an internal JAX value

In [ ]:
print ("First call: ", jit(impure_saves_global)(5.))

In [ ]:
jnp.array([1])

In [ ]:
st=dict(even=0, odd=0)
@jit
def pure_uses_internal_state(x):
  state = st#dict(even=0, odd=0)
  for i in range(10):
    state['even' if i % 2 == 0 else 'odd'] += x
  return state['even'] + state['odd']

print(jit(pure_uses_internal_state)(7.))

In [ ]:
import jax.numpy as jnp
import jax.lax as lax
from jax import make_jaxpr

# lax.fori_loop
array = jnp.arange(10)
print(lax.fori_loop(0, 10, lambda i,x: x+array[i], 0)) # expected result 45
iterator = iter(range(10))
print(lax.fori_loop(0, 10, lambda i,x: x+next(iterator), 0)) # unexpected result 0

In [ ]:
help(lax.scan)

In [ ]:
# lax.scan
def func11(arr, extra):
    ones = jnp.ones(arr.shape)
    def body(carry, aelems):
        ae1, ae2 = aelems
        return (carry + ae1 * ae2 + extra, carry)
    return lax.scan(body, 0., (arr, ones))
make_jaxpr(func11)(jnp.arange(16), 5.)
# make_jaxpr(func11)(iter(range(16)), 5.) # throws error

In [ ]:
func11(jnp.arange(16), 5.)

In [ ]:
arr=jnp.arange(16)
extra=5
ones = jnp.ones(arr.shape)
def body(state, aelems):
    print('state:',state)
    print('aelems:',aelems)
    ae1, ae2 = aelems
    return (state + ae1 * ae2 + extra, state)
lax.scan(body, 0., (arr, ones))


In [ ]:
help(lax.cond)

In [ ]:
# lax.cond
print('arr:',arr)
# lax.cond(arr%2==0, lambda x: x[0]+1, lambda x: x[0]-1, (arr, ones))
arr%2==0

In [ ]:
b=arr%2==0
print(b)
jnp.where(b,size=40,fill_value=-1)

In [ ]:
numpy_array = np.zeros((3,3), dtype=np.float32)
print("original array:")
print(numpy_array)

# In place, mutating update
numpy_array[1, :] = 1.0
print("updated array:")
print(numpy_array)

In [ ]:
arr=jnp.array((3,4))
arr

In [ ]:
x = random.normal(key, (3,4))
x

In [ ]:
x.at[jnp.where(x>0.2)].set(1)

In [ ]:
import jax
def my_log(x):
  return jnp.where(x > 0., jnp.log(x+1e-2), jnp.log(x+1e-3))

jax.grad(my_log)(0.) # ==> NaN

In [ ]:
jax.make_jaxpr(jax.grad(my_log))(0.)

In [ ]:
from jax.experimental import io_callback
from functools import partial

global_rng = np.random.default_rng(0)

def host_side_random_like(x):
  """Generate a random array like x using the global_rng state"""
  # We have two side-effects here:
  # - printing the shape and dtype
  # - calling global_rng, thus updating its state
#   print(f'generating {x.dtype}{x}')
  return global_rng.uniform(size=x.shape).astype(x.dtype)

@jax.jit
def numpy_random_like(x):
  return io_callback(host_side_random_like, x, x)

x = jnp.zeros(5)
numpy_random_like(x)

In [ ]:
help(io_callback)

In [ ]:
jax_array = jnp.zeros((3,3), dtype=jnp.float32)
updated_array = jax_array.at[1, :].set(1.0)
print("updated array:\n", updated_array)

In [ ]:
print("original array:")
jax_array = jnp.ones((5, 6))
print(jax_array)

new_jax_array = jax_array.at[::2, 3:].add(7.)
print("new array post-addition:")
print(new_jax_array)

In [ ]:
jnp.arange(10.0).at[11].set(1)

In [ ]:
jnp.arange(10.0).at[11].get(mode='fill', fill_value=jnp.nan)

In [ ]:
@jit
def permissive_sum(x):
  return jnp.sum(x)

x = jnp.array(list(range(10)))
permissive_sum(x)

In [ ]:
%timeit permissive_sum(x)

In [ ]:
help(random.split)

In [ ]:
def f(x):
    return  jnp.log(x)

x=jax.random.normal(key,(1000,))
print(f(x).shape)
%timeit f(x)

In [ ]:
@jit
def f(x):
    return  jnp.log(x)

x=jax.random.normal(key,(1000,))
print(f(x).shape)
%timeit f(x)

In [ ]:
@jit
def f(x):
  for i in range(3):
    x = 2 * x
  return x

print(f(3))

In [ ]:
jnp.prod(jnp.array(x.shape))

In [ ]:
@jit
def g(x):
  s=np.prod(np.array(x.shape))
  print(s)
  return jnp.ones(s)

%timeit -n1 -r1 print(g(jnp.array([[1., 2., 3.]])))
%timeit -n1 -r1 g(jnp.array([[1., 2., 3.]]))

In [ ]:
%timeit -n1 -r1 print(g(jnp.array([1., 2., 3.,4])))
%timeit -n1 -r1 g(jnp.array([1., 2., 3.,4]))

In [ ]:
@partial(jit,static_argnums=(1,))
def f(x, n):
  y = 0.
  for i in range(n):
    y = y + x[i]
  return y

#f = jit(f, static_argnums=(1,))
a=jnp.array([2., 3., 4.])
f(a, 2)
%timeit f(a, 2)

In [ ]:
@jit
def f(x, y):
   a = x * y
   b = (x + y) / (x - y)
   c = a + 2
   return a + b * c


x = jnp.array([2., 0.])

y = jnp.array([3., 0.])

f(x, y)


In [ ]:
import jax
import jax.numpy as jnp
import jax.profiler

def func1(x):
  return jnp.tile(x, 10) * 0.5

def func2(x):
  y = func1(x)
  return y, jnp.tile(x, 10) + 1

x = jax.random.normal(jax.random.PRNGKey(42), (1000, 1000))
y, z = func2(x)

z.block_until_ready()

jax.profiler.save_device_memory_profile("memory.prof")

In [ ]:
!ls -alh

In [ ]:
!pprof 
!go

In [ ]:
import jax
import jax.numpy as jnp

@jax.jit
def f(x):
  jax.debug.print("🤯 {x} 🤯", x=x)
  y = jnp.sin(x)
  jax.debug.breakpoint()
  jax.debug.print("🤯 {y} 🤯", y=y)
  return y

f(2.)


In [ ]:
import jax
from jax import lax
def breakpoint_if_nonfinite(x):
  is_finite = jnp.isfinite(x).all()
  def true_fn(x):
    pass
  def false_fn(x):
    jax.debug.breakpoint()
  lax.cond(is_finite, true_fn, false_fn, x)

@jax.jit
def f(x, y):
  z = x / y
  breakpoint_if_nonfinite(z)
  return z
f(2., 0.) # ==> Pauses during execution!

In [ ]:
xs = jnp.arange(3.)

def f(x):
  jax.debug.print("x: {}", x)
  y = jnp.sin(x)
  jax.debug.print("y: {}", y)
  return y
print('v1',jax.vmap(f)(xs))
# Prints: x: 0.0
#         x: 1.0
#         x: 2.0
#         y: 0.0
#         y: 0.841471
#         y: 0.9092974
print('v2',jax.lax.map(f, xs))
# Prints: x: 0.0
#         y: 0.0
#         x: 1.0
#         y: 0.841471
#         x: 2.0
#         y: 0.9092974

In [ ]:
from jax.experimental import checkify
import jax
import jax.numpy as jnp

def f(x, i):
  checkify.check(i >= 0, f"index needs to be non-negative, got {i}")
  y = x[i]
  z = jnp.sin(y)
  return z

jittable_f = checkify.checkify(f)

err, z = jax.jit(jittable_f)(jnp.ones((5,)), -2)
print(err.get())
# >> index needs to be non-negative, got -2! (check failed at <...>:6 (f))

In [ ]:
errors = checkify.user_checks | checkify.index_checks | checkify.float_checks
checked_f = checkify.checkify(f, errors=errors)

err, z = checked_f(jnp.ones((5,)), 100)
err.throw()
# ValueError: out-of-bounds indexing at <..>:7 (f)

err, z = checked_f(jnp.ones((5,)), -1)
err.throw()
# ValueError: index needs to be non-negative! (check failed at <…>:6 (f))

err, z = checked_f(jnp.array([jnp.inf, 1]), 0)
err.throw()
# ValueError: nan generated by primitive sin at <...>:8 (f)

err, z = checked_f(jnp.array([5, 1]), 0)
err.throw()  # if no error occurred, throw does nothing!

In [ ]:
import jax
import jax.numpy as jnp
import numpy as np

def f_host(x):
  return np.sin(x).astype(x.dtype)

def f(x):
  return jax.pure_callback(f_host, x, x)

x = jnp.arange(5.0)
f(x)

In [ ]:
jax.jit(f)(x)

In [ ]:
jax.vmap(f)(x)

In [ ]:
def body_fun(_, x):
  return _+1, f(x)
jax.lax.scan(body_fun, 0, jnp.arange(5.0))

In [ ]:
%xmode minimal
jax.grad(f)(x)

In [ ]:
jnp.asarray(2)

In [ ]:
jnp.array(2)

In [ ]:
from jax import jit
from functools import partial
@partial(jit, static_argnums=1)
def func(x, axis):
    return x.min(axis)

In [ ]:
func(jnp.arange(4), 0)  

In [ ]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random

In [ ]:
# A helper function to randomly initialize weights and biases
# for a dense neural network layer
def random_layer_params(m, n, key, scale=1e-2):
  w_key, b_key = random.split(key)
  return scale * random.normal(w_key, (n, m)), scale * random.normal(b_key, (n,))

# Initialize all layers for a fully-connected neural network with sizes "sizes"
def init_network_params(sizes, key):
  keys = random.split(key, len(sizes))
  return [random_layer_params(m, n, k) for m, n, k in zip(sizes[:-1], sizes[1:], keys)]

layer_sizes = [784, 512, 512, 10]
step_size = 0.01
num_epochs = 10
batch_size = 128
n_targets = 10
params = init_network_params(layer_sizes, random.PRNGKey(0))

In [ ]:
from jax.scipy.special import logsumexp

def relu(x):
  return jnp.maximum(0, x)

def predict(params, image):
  # per-example predictions
  activations = image
  for w, b in params[:-1]:
    outputs = jnp.dot(w, activations) + b
    activations = relu(outputs)
  
  final_w, final_b = params[-1]
  logits = jnp.dot(final_w, activations) + final_b
  return logits - logsumexp(logits)  #log(p)

In [ ]:
logsumexp(np.array([-2000,-1.,0,1,20]))

In [ ]:
def lx(logits):
    return logits - logsumexp(logits)
lx(jnp.array([-2000,-1.,0,1,20]))

In [ ]:
# This works on single examples
random_flattened_image = random.normal(random.PRNGKey(1), (28 * 28,))
preds = predict(params, random_flattened_image)
print(preds.shape)

In [ ]:
# Doesn't work with a batch
random_flattened_images = random.normal(random.PRNGKey(1), (10, 28 * 28))
try:
  preds = predict(params, random_flattened_images)
except TypeError:
  print('Invalid shapes!')

In [ ]:
# Let's upgrade it to handle batches using `vmap`

# Make a batched version of the `predict` function
batched_predict = vmap(predict, in_axes=(None, 0))

# `batched_predict` has the same call signature as `predict`
batched_preds = batched_predict(params, random_flattened_images)
print(batched_preds.shape)

In [ ]:
x=jnp.array([1,2,3])
k=9
jnp.array(x[:, None] == jnp.arange(k), jnp.float32)

In [ ]:
def one_hot(x, k, dtype=jnp.float32):
  """Create a one-hot encoding of x of size k."""
  return jnp.array(x[:, None] == jnp.arange(k), dtype)
  
def accuracy(params, images, targets):
  target_class = jnp.argmax(targets, axis=1)
  predicted_class = jnp.argmax(batched_predict(params, images), axis=1)
  return jnp.mean(predicted_class == target_class)

def loss(params, images, targets):
  preds = batched_predict(params, images)
  return -jnp.mean(preds * targets)

@jit
def update(params, x, y):
  grads = grad(loss)(params, x, y)
  return [(w - step_size * dw, b - step_size * db)
          for (w, b), (dw, db) in zip(params, grads)]

In [ ]:
import tensorflow as tf
# Ensure TF does not see GPU and grab all GPU memory.
tf.config.set_visible_devices([], device_type='GPU')

import tensorflow_datasets as tfds

data_dir = '/tmp/tfds'

# Fetch full datasets for evaluation
# tfds.load returns tf.Tensors (or tf.data.Datasets if batch_size != -1)
# You can convert them to NumPy arrays (or iterables of NumPy arrays) with tfds.dataset_as_numpy
mnist_data, info = tfds.load(name="mnist", batch_size=-1, data_dir=data_dir, with_info=True)
mnist_data = tfds.as_numpy(mnist_data)
train_data, test_data = mnist_data['train'], mnist_data['test']
num_labels = info.features['label'].num_classes
h, w, c = info.features['image'].shape
num_pixels = h * w * c

# Full train set
train_images, train_labels = train_data['image'], train_data['label']
train_images = jnp.reshape(train_images, (len(train_images), num_pixels))
train_labels = one_hot(train_labels, num_labels)

# Full test set
test_images, test_labels = test_data['image'], test_data['label']
test_images = jnp.reshape(test_images, (len(test_images), num_pixels))
test_labels = one_hot(test_labels, num_labels)

In [ ]:
print('Train:', train_images.shape, train_labels.shape)
print('Test:', test_images.shape, test_labels.shape)

In [ ]:
import time

def get_train_batches():
  # as_supervised=True gives us the (image, label) as a tuple instead of a dict
  ds = tfds.load(name='mnist', split='train', as_supervised=True, data_dir=data_dir)
  # You can build up an arbitrary tf.data input pipeline
  ds = ds.batch(batch_size).prefetch(1)
  # tfds.dataset_as_numpy converts the tf.data.Dataset into an iterable of NumPy arrays
  return tfds.as_numpy(ds)

for epoch in range(num_epochs):
  start_time = time.time()
  for x, y in get_train_batches():
    x = jnp.reshape(x, (len(x), num_pixels))
    y = one_hot(y, num_labels)
    params = update(params, x, y)
  epoch_time = time.time() - start_time

  train_acc = accuracy(params, train_images, train_labels)
  test_acc = accuracy(params, test_images, test_labels)
  print("Epoch {} in {:0.2f} sec".format(epoch, epoch_time))
  print("Training set accuracy {}".format(train_acc))
  print("Test set accuracy {}".format(test_acc))

In [ ]:
[[e.shape for e in p] for p in params]

In [ ]:
from flax import linen as nn

In [ ]:

class CNN(nn.Module):
  """A simple CNN model."""

  @nn.compact
  def __call__(self, x):
    x = nn.Conv(features=32, kernel_size=(3, 3))(x)
    x = nn.relu(x)
    x = nn.avg_pool(x, window_shape=(2, 2), strides=(2, 2))
    x = nn.Conv(features=64, kernel_size=(3, 3))(x)
    x = nn.relu(x)
    x = nn.avg_pool(x, window_shape=(2, 2), strides=(2, 2))
    x = x.reshape((x.shape[0], -1))  # flatten
    x = nn.Dense(features=256)(x)
    x = nn.relu(x)
    x = nn.Dense(features=10)(x)
    return x

In [1]:
!pip install keras-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 924.6/924.6 kB 11.8 MB/s eta 0:00:0000:0100:01


# JAX

In [2]:
import sys
if '/kaggle/input/kaggle-utils' not in sys.path:
    sys.path.append('/kaggle/input/kaggle-utils')
import utils
import importlib 
importlib.reload(utils)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

Using JAX backend.


<module 'utils' from '/kaggle/input/kaggle-utils/utils.py'>

In [4]:
import os
import jax
import numpy as np
import tensorflow as tf
import keras_core as keras

from jax.experimental import mesh_utils
from jax.sharding import Mesh
from jax.sharding import NamedSharding
from jax.sharding import PartitionSpec
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random
key = random.PRNGKey(0)


def get_model():
    # Make a simple 2-layer densely-connected neural network.
    inputs = keras.Input(shape=(28,28,1))
    x = keras.layers.Reshape((28*28,))(inputs)
    x = keras.layers.Dense(256, activation="relu")(x)
    x = keras.layers.Dense(256, activation="relu")(x)
    outputs = keras.layers.Dense(10)(x)
    model = keras.Model(inputs, outputs)
    return model


def get_datasets():
    # Load the data and split it between train and test sets
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

    # Scale images to the [0, 1] range
    x_train = x_train.astype("float32")
    x_test = x_test.astype("float32")
    # Make sure images have shape (28, 28, 1)
    x_train = np.expand_dims(x_train, -1)
    x_test = np.expand_dims(x_test, -1)
    print("x_train shape:", x_train.shape)
    print(x_train.shape[0], "train samples")
    print(x_test.shape[0], "test samples")

    # Create TF Datasets
    train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    eval_data = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    return train_data, eval_data

num_epochs = 20
batch_size = 64 * 8

train_data, eval_data = get_datasets()
train_data = train_data.batch(batch_size, drop_remainder=True)

devices = utils.get_devices()
test_shard_data=random.normal(key, (batch_size, 128*128), dtype=jnp.float32)
data_sharding = utils.get_data_sharding(devices,test_shard_data)

model = get_model()
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = keras.optimizers.Adam(1e-3)
utils.set_model_suit(model,loss_fn,optimizer)

(one_batch, one_batch_labels) = next(iter(train_data))
model.build(one_batch)
optimizer.build(model.trainable_variables)
model.summary()

train_state = utils.get_replicated_train_state(devices,model,optimizer)

# Custom training loop
for epoch in range(num_epochs):
    import time
    start=time.time()
    data_iter = iter(train_data)
    for data in data_iter:
        x, y = data
        sharded_x = jax.device_put(x.numpy(), data_sharding)
        loss_value, train_state = utils.train_step(train_state, sharded_x, y.numpy())
    print("Epoch", epoch, "loss:", loss_value,'cost:',time.time()-start)

# Post-processing model state update to write them back into the model
trainable_variables, non_trainable_variables, optimizer_variables = train_state
for variable, value in zip(model.trainable_variables, trainable_variables):
    variable.assign(value)
for variable, value in zip(model.non_trainable_variables, non_trainable_variables):
    variable.assign(value)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Running on 1 devices: [gpu(id=0)]
Data sharding...
test_shard_data: (512, 16384)


                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                     GPU 0                                      
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                

Data sharding over!


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 28, 28, 1)         │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ reshape_1 (Reshape)             │ (None, 784)               │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_3 (Dense)                 │ (None, 256)               │    200,960 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_4 (Dense)                 │ (None, 256)               │     65,792 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_5 (Dense)                 │ (None, 10)                │      2,570 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 269,322 (1.03 MB)

 Trainable params: 269,322 (1.03 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 0 loss: 0.9693474 cost: 51.677998304367065
Epoch 1 loss: 0.47046852 cost: 0.4756608009338379
Epoch 2 loss: 0.25392938 cost: 0.5402429103851318
Epoch 3 loss: 0.11509393 cost: 0.29418468475341797
Epoch 4 loss: 0.12776947 cost: 0.27455568313598633
Epoch 5 loss: 0.107299134 cost: 0.30698108673095703
Epoch 6 loss: 0.043443 cost: 0.2800438404083252
Epoch 7 loss: 0.104660384 cost: 0.29596829414367676
Epoch 8 loss: 0.01719129 cost: 0.2734522819519043
Epoch 9 loss: 0.027290137 cost: 0.3023505210876465
Epoch 10 loss: 0.0095254015 cost: 0.2686445713043213
Epoch 11 loss: 0.032697883 cost: 0.2578747272491455
Epoch 12 loss: 0.029653922 cost: 0.26798057556152344
Epoch 13 loss: 0.035819754 cost: 0.33320116996765137
Epoch 14 loss: 0.023044964 cost: 0.26282453536987305
Epoch 15 loss: 0.0038949219 cost: 0.25931549072265625
Epoch 16 loss: 0.13145132 cost: 0.31117773056030273
Epoch 17 loss: 0.01595957 cost: 0.2969954013824463
Epoch 18 loss: 0.054846402 cost: 0.2645421028137207
Epoch 19 loss: 0.060465